In [18]:
# Download the surnames here: https://www2.census.gov/topics/genealogy/2010surnames/
# Here I converted the Excel file to csv as the input
# I'm practicing Spark2.2 here, you don't have to use Spark

from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import string

In [3]:
spark = SparkSession.builder.master("local")\
        .appName("Anomalies Detection")\
        .config("spark.some.config.option", "some-value")
        .getOrCreate()

In [23]:
df = spark.read.csv("Names_2010Census_Top1000.csv", header=True)
df.show(n=7)

+--------+----+-----------------+---------------------------------+---------------------+------------------------------------------+--------------------------------------------------------------+-----------------------------------------------------------------------------------------+----------------------------------------------------------------------+------------------------------------------------+---------------------------------+
| SURNAME|RANK|FREQUENCY (COUNT)|PROPORTION PER 100,000 POPULATION|CUMULATIVE PROPORTION|PERCENT NON-HISPANIC OR LATINO WHITE ALONE|PERCENT NON-HISPANIC OR LATINO BLACK OR AFRICAN AMERICAN ALONE|PERCENT NON-HISPANIC OR LATINO ASIAN AND NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE|PERCENT NON-HISPANIC OR LATINO AMERICAN INDIAN AND ALASKA NATIVE ALONE|PERCENT NON-HISPANIC OR LATINO TWO OR MORE RACES|PERCENT HISPANIC OR LATINO ORIGIN|
+--------+----+-----------------+---------------------------------+---------------------+-------------------------------

In [24]:
df.select('SURNAME').show(n=7)

+--------+
| SURNAME|
+--------+
|   SMITH|
| JOHNSON|
|WILLIAMS|
|   BROWN|
|   JONES|
|  GARCIA|
|  MILLER|
+--------+
only showing top 7 rows



In [25]:
# define udf
## the method is simply, just add a to z before each surname, one by one

alph_lst = string.ascii_lowercase[:26]

def get_namelst(surname):
    results = [c + surname for c in alph_lst]
    return results

convertUDF = udf(lambda r: get_namelst(r), ArrayType(StringType()))
newdf = df.withColumn("namelst", convertUDF(df.SURNAME))

In [26]:
newdf.select('SURNAME', 'namelst').show(n=7)

+--------+--------------------+
| SURNAME|             namelst|
+--------+--------------------+
|   SMITH|[aSMITH, bSMITH, ...|
| JOHNSON|[aJOHNSON, bJOHNS...|
|WILLIAMS|[aWILLIAMS, bWILL...|
|   BROWN|[aBROWN, bBROWN, ...|
|   JONES|[aJONES, bJONES, ...|
|  GARCIA|[aGARCIA, bGARCIA...|
|  MILLER|[aMILLER, bMILLER...|
+--------+--------------------+
only showing top 7 rows

